In [ ]:
import numpy as np
import numba
from scipy.optimize import minimize, OptimizeResult
from scipy.integrate import solve_ivp
from plotting import plot
from functools import lru_cache

In [ ]:
STANDARD_GRAV = 9.80665
CdA = 0.3 * 0.01
rho = 1.25
g0 = STANDARD_GRAV
#a0 = -0.001
#b0 = 3.0

params_ball = (rho, CdA)

x0 = np.array([0,0,10,10])

In [ ]:
@numba.njit
def dynamics(t, x, params):
    rho, CdA = params
    dx = np.zeros_like(x)
    v_sq = x[2]**2 + x[3]**2
    drag = 0.5 * rho * CdA * v_sq
    dx[0] = x[2]
    dx[1] = x[3]
    dx[2] = - drag * x[2] / np.sqrt(v_sq)
    dx[3] =  - drag * x[3] / np.sqrt(v_sq) - g0
    return dx

In [ ]:
decision_init_guess = (
    0,0 ,float(np.deg2rad(60)), 100, 10,
    500,300 ,float(np.deg2rad(0)), 50, 10
    )

In [ ]:
@lru_cache(maxsize=128, typed=True)
def traj_rollout(decision_var: tuple, params: tuple) -> OptimizeResult:

    pos_x, pos_y, pitch, vel, t_terminal = decision_var
    v_x = vel* np.cos(pitch)
    v_y = vel* np.sin(pitch)
    x0 = np.array([pos_x, pos_y, v_x, v_y])
    sol = solve_ivp(
        dynamics, 
        t_span=[0.0, t_terminal], 
        y0=x0,    
        args=(params,)
    )
    return sol  

In [ ]:
def objective(decision_var: tuple, params: tuple):
        pos_x, pos_y, pitch, vel, t_terminal, pos2_x, pos2_y, pitch2, vel2, t_terminal2 = decision_var
        cost =  vel **2
        return cost

def jac_objective(decision_var: tuple, params: tuple):
        pos_x, pos_y, pitch, vel, t_terminal, pos2_x, pos2_y, pitch2, vel2, t_terminal2 = decision_var
        print("jac", vel**2)
        return 2*vel

In [ ]:
def dynamics_knot_constrant(decision_var: tuple, params: tuple):
        pos_x, pos_y, pitch, vel, t_terminal, pos2_x, pos2_y, pitch2, vel2, t_terminal2 = decision_var
        # Segment 1
        sol1 = traj_rollout((pos_x, pos_y, pitch, vel, t_terminal), params)
        # Segment 2
        sol2 = traj_rollout((pos2_x, pos2_y, pitch2, vel2, t_terminal2), params)

        defects =  sol2.y[:,0] - sol1.y[:,-1]
        terminal_defect =  np.array([1000, 0]) - sol2.y[0:2,-1]
        return np.concatenate([defects, terminal_defect])

In [ ]:
bounds = [
    (0.,0.), # segment 1 x0
    (0.,0.), # segment 1 y0
    (0., np.deg2rad(60)), # initial pitch bound 
    (10.,1000.), # initial speed bound 
    (0., 30.), # terminal time bound 
    (0.,np.inf), # x cant be smaller than first phase start
    (0.,np.inf), # y cant be less than zero
    (0., 0.), # initial pitch bound, segment boundry at 0 pitch. i.e max height
    (0.,300.), # initial speed bound, segment 2 
    (0., 30.) # terminal time bound, segment 2 
]

constraints = [{'type': 'eq', 'fun': dynamics_knot_constrant, 'args':(params_ball,) },]


In [ ]:
result = minimize(
    objective, 
    decision_init_guess, 
    # jac= jac_objective,
    method='SLSQP', 
    bounds=bounds, 
    constraints=constraints,
    args=(params_ball,)
)
result

In [ ]:
pos_x, pos_y, pitch, vel, t_terminal, pos2_x, pos2_y, pitch2, vel2, t_terminal2 = result.x
# Segment 1
sol1 = traj_rollout((pos_x, pos_y, pitch, vel, t_terminal), params_ball)
# Segment 2
sol2 = traj_rollout((pos2_x, pos2_y, pitch2, vel2, t_terminal2), params_ball)


In [ ]:
plot(
    sol1.t,[sol1.y[0], sol1.y[1]], 
    y2 = [sol1.y[2], sol1.y[3]],
    title="Time vs States, Segment 2", 
    xlabel="Time", 
    ylabel=("Pos", "Vel"),
    trace_names=("pos_x", "pos_y", "vel_x", "vel_y")
    )

In [ ]:
plot(
    sol2.t + sol1.t[-1],[sol2.y[0], sol2.y[1]], 
    y2 = [sol2.y[2], sol2.y[3]],
    title="Time vs States Segment 1", 
    xlabel="Time", 
    ylabel=("Pos", "Vel"),
    trace_names=("pos_x", "pos_y", "vel_x", "vel_y")
    )